In [2]:
pip install pyprojroot

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install chromadb

  Installing build dependencies ... one
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached bcrypt-4.2.1-cp39-abi3-macosx_10_12_universal2.whl.metadata (9.8 kB)
  Using cached rich-13.9.4-py3-none-any.whl.metadata (18 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 8.2 MB/s eta 0:00:00
Using cached bcrypt-4.2.1-cp39-abi3-macosx_10_12_universal2.whl (489 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 24.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 46.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.5 MB/s eta 0:00:00
Using cached rich-13.9.4-py3-none-any.whl (242 kB)
Using cached markdown_it_py-3.0.0-py3-none-any.whl (87

In [2]:
from pyprojroot import here
import pandas as pd
import chromadb
from FlagEmbedding import FlagModel

In [52]:
chroma_client = chromadb.PersistentClient(path=str(here("data/chroma_itam")))

In [53]:
collection = chroma_client.create_collection(name="itam_hw")

In [54]:
file_dir = here("data/itam_files/itam_hw.csv")
df = pd.read_csv(file_dir, nrows=100)

In [55]:
df

,Asset_ID,Asset_Name,Asset_Type,Manufacturer,Model,Purchase_Date,Warranty_Expiry,Assigned_To,Status
0,HW001,It Device,Hardware,Lenovo,Model-4915,2022-07-07,2027-12-29,William Jackson,Retired
1,SW002,Source Software,Software,SAP,Version-82.2,2022-11-06,2027-04-17,Ashok Arulsamy,Active
2,HW003,Green Device,Hardware,HP,Model-8953,2020-09-28,2027-03-16,Jennifer Adams,Retired
3,SW004,Human Software,Software,Oracle,Version-38.7,2023-10-04,2028-10-01,HR Specialist,Expired
4,HW005,Sound Device,Hardware,Apple,Model-2092,2021-03-29,2029-07-28,Mrs. Grace Molina,Retired
...,...,...,...,...,...,...,...,...,...
95,HW096,Middle Device,Hardware,Asus,Model-6021,2022-12-22,2025-12-08,Dr. Heather Wong,Available
96,SW097,Official Software,Software,Oracle,Version-21.6,2024-06-29,2028-07-18,System Administrator,Expired
97,HW098,Public Device,Hardware,Dell,Model-9253,2024-01-24,2029-09-06,Angela Anderson,Available
98,HW099,Book Device,Hardware,Dell,Model-8047,2020-09-07,2027-08-18,Matthew Townsend,Retired


In [56]:
embedding_model = FlagModel('BAAI/bge-large-zh-v1.5',
                  #query_instruction_for_retrieval="largest animal on the planet",
                  # use_fp16=True
                  )

In [57]:
docs = []
metadatas = []
ids = []
embeddings = []
for index, row in df.iterrows():
    output_str = ""
    # Treat each row as a separate chunk
    for col in df.columns:
        output_str += f"{col}: {row[col]},\n"
    response = embedding_model.encode(output_str)
    embeddings.append(response)
    docs.append(output_str)
    metadatas.append({"source": "itam_hw"})
    ids.append(f"id{index}")

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [58]:
docs

['Asset_ID: HW001,\nAsset_Name: It Device,\nAsset_Type: Hardware,\nManufacturer: Lenovo,\nModel: Model-4915,\nPurchase_Date: 2022-07-07,\nWarranty_Expiry: 2027-12-29,\nAssigned_To: William Jackson,\nStatus: Retired,\n',
 'Asset_ID: SW002,\nAsset_Name: Source Software,\nAsset_Type: Software,\nManufacturer: SAP,\nModel: Version-82.2,\nPurchase_Date: 2022-11-06,\nWarranty_Expiry: 2027-04-17,\nAssigned_To: Ashok Arulsamy,\nStatus: Active,\n',
 'Asset_ID: HW003,\nAsset_Name: Green Device,\nAsset_Type: Hardware,\nManufacturer: HP,\nModel: Model-8953,\nPurchase_Date: 2020-09-28,\nWarranty_Expiry: 2027-03-16,\nAssigned_To: Jennifer Adams,\nStatus: Retired,\n',
 'Asset_ID: SW004,\nAsset_Name: Human Software,\nAsset_Type: Software,\nManufacturer: Oracle,\nModel: Version-38.7,\nPurchase_Date: 2023-10-04,\nWarranty_Expiry: 2028-10-01,\nAssigned_To: HR Specialist,\nStatus: Expired,\n',
 'Asset_ID: HW005,\nAsset_Name: Sound Device,\nAsset_Type: Hardware,\nManufacturer: Apple,\nModel: Model-2092,\nPu

In [59]:
print(metadatas)
print(ids)

[{'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source': 'itam_hw'}, {'source':

In [60]:
embeddings[0][:10]

array([-0.02473 ,  0.01573 , -0.04657 ,  0.01125 , -0.01888 ,  0.0994  ,
        0.000337,  0.008026,  0.02554 ,  0.03363 ], dtype=float16)

In [61]:
collection.add(
    documents=docs,
    metadatas=metadatas,
    embeddings=embeddings,
    ids=ids
)

In [35]:
print("Number of vectors in vectordb:", collection.count())

Number of vectors in vectordb: 100


In [62]:
query_texts = "More about Ashok Arulsamy"
query_embeddings = embedding_model.encode(query_texts)

In [63]:
vectordb = chroma_client.get_collection(name="itam_hw")
vectordb.count()

100

In [65]:
results = vectordb.query(
    query_embeddings = query_embeddings,
    n_results=1 #top_k
)

results

{'ids': [['id1']],
 'embeddings': None,
 'documents': [['Asset_ID: SW002,\nAsset_Name: Source Software,\nAsset_Type: Software,\nManufacturer: SAP,\nModel: Version-82.2,\nPurchase_Date: 2022-11-06,\nWarranty_Expiry: 2027-04-17,\nAssigned_To: Ashok Arulsamy,\nStatus: Active,\n']],
 'uris': None,
 'data': None,
 'metadatas': [[{'source': 'itam_hw'}]],
 'distances': [[1.1954249143600464]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}